# Determine the frequency of duplicate scores across several osu random dump
**Contributors:** Victor Lin

**Achievement:** The frequency of duplicate scores (scores from the same player, beatmap, and mods) was found to be 778, or ~.00078% of the 10M scores present across all dumps. Duplicates can be safely ignored for the data cleaning process.

**Requirements:**

1. env must be set with MySQL DB & Mongo log-in info
2. MySQL DB has imported osu standard dumps (random or top) from https://data.ppy.sh/

## Introduction
Isolated osu random dumps contain 10k unique players. However, there can be player crossover *between* dumps. This leads to the possibility of duplicate scores, where a player in an older dump updates their score for a particular beatmap+mod, resulting in a repeat in the new dump.

The SQL pipeline to pinpoint the frequency of these duplicates falls under 2 steps:

1. Reduce Player Search Space

2. Calculate Duplicate Frequency

In [ ]:
import sys
sys.path.append('../..')
import pymysql
import pandas as pd
from exploration.config import sql_inst

In [ ]:
random_dump_titles = [
    "osu_random_2020_08",
    "osu_random_2020_09",
    "osu_random_2020_10",
    "osu_random_2020_11",
    "osu_random_2020_12",
    "osu_random_2021_01"
]

## 1) Reduce Search Space
Union all osu_user_stats tables to identify only the players with crossover. No need to search other players, as they cannot have duplicates.

In [ ]:
def find_repeat_users(dump_titles):
    template = "(SELECT user_id FROM {}.osu_user_stats)".format
    QUERY_UNION_USER_ID = '\nUNION ALL\n'.join(map(template, random_dump_titles))

    with sql_inst.cursor() as cursor:
        cursor.execute(
            f"""
            SELECT user_id
            FROM (
                {QUERY_UNION_USER_ID}
            )as USERS_INCLUDING_DUPE
            GROUP BY user_id
            HAVING COUNT(*) > 1
            ORDER BY COUNT(*) DESC 
            """
        )

        return tuple(row[0] for row in cursor)

REPEAT_USERS_TUPLE = find_repeat_users(random_dump_titles)

## 2) Frequency Calculation
Only for osu highscores from the players identified, group scores by unique (player, beatmap, mod) combinations. Accumulate the frequency of groups > 1 (aka duplicate scores)

In [ ]:
def get_repeat_freq_table(repeat_users):
    template = ("""
    SELECT score_id, user_id, beatmap_id, enabled_mods
            FROM {}.osu_scores_high
            WHERE
            user_id IN """ + str(REPEAT_USERS_TUPLE)
    ).format

    QUERY_USERS_UNION_SCORES = '\nUNION\n'.join(map(template, random_dump_titles))

    query = f"""
            SELECT B.num_dumps, COUNT(*) as freq FROM (
                SELECT COUNT(*) as num_dumps FROM (
                    {QUERY_USERS_UNION_SCORES}
                ) as A
                GROUP BY A.user_id, A.beatmap_id, A.enabled_mods
            ) as B
            WHERE B.num_dumps > 1
            GROUP BY B.num_dumps
            """

    return pd.read_sql(query, sql_inst)

get_repeat_freq_table(REPEAT_USERS_TUPLE)

## Interpretation
The above table shows score repeats for 2 dumps were found 778 times out of all 6 osu random dumps. There were no scores that were found across 3 or more dumps. This makes sense, as the probability of each additional dump update should exponentially decreases.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f93d0822-db5a-47ef-9a78-57b8adfbeb20' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>